In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-07-05 09:38:09--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-07-05 09:38:09 (19.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open('input.txt','r',encoding='utf-8') as f:
  text=f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [ ]:
text[:8]

'First Ci'

In [ ]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
stoi={ch:i for i,ch in enumerate(chars)}
itos={i:ch for i,ch in enumerate(chars)}
encode = lambda s : [stoi[c] for c in s] # encoder: take a string output a list of integers
decode = lambda l : ''.join([itos[i] for i in l]) #decoder takes a list of integers , outputs a string

In [ ]:
encode("Hello")
decode(encode("Hello"))

'Hello'

In [ ]:
import torch
data=torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:20])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56])


In [ ]:
# splitting the dataset into train and validation sets
n=int(0.9*len(text))
train_data=data[:n]
val_data=data[n:]

In [ ]:
val_data

tensor([12,  0,  0,  ..., 45,  8,  0])

In [ ]:
block_size=8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x=train_data[:block_size]
y=train_data[1:block_size+1] # Corrected the slicing for y
for t in range(block_size):
  context=x[:t+1]
  target=y[t]

  # Convert tensors to lists of integers before decoding
  print(f"When context is {decode(context.tolist())} , the target is {decode([target.tolist()])}\n")

When context is F , the target is i

When context is Fi , the target is r

When context is Fir , the target is s

When context is Firs , the target is t

When context is First , the target is  

When context is First  , the target is C

When context is First C , the target is i

When context is First Ci , the target is t



In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data

    ix=torch.randint(len(data)-block_size,(batch_size,))
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x,y

xb,yb=get_batch('train')

print('inputs:')
print(xb.shape)
print(xb)

print('targets:')
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context=xb[b,:t+1]
        target=yb[b,t]

        # Convert tensors to lists of integers before decoding
        print(f"When context is {decode(context.tolist())} , the target is {decode([target.tolist()])}\n")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
When context is L , the target is e

When context is Le , the target is t

When context is Let , the target is '

When context is Let' , the target is s

When context is Let's , the target is  

When context is Let's  , the target is h

When context is Let's h , the target is e

When context is Let's he , the target is a

When context is f , the target is o

When context is fo , the target is r

When context is for , the target is  

When context is for  , the target is t

When context is for t , the target is h

When context is for th , the target is a

When context is for tha , the target is t

Whe

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self,vocab_size):

    super().__init__()

    # each token directly reads off the logits for the next token from the embedding table

    self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)


  def forward(self,idx,target=None):

    # idx and target are both (B,T) tensors of integers

    logits=self.token_embedding_table(idx) # (B,T,C)

    if target is None:
      loss=None
    else:
      B,T,C=logits.shape
      logits=logits.view(B*T,C)
      target=target.view(B*T)
      loss=F.cross_entropy(logits,target)

    return logits ,loss


  def generate(self,idx,max_new_tokens):
    # idx is (B,T) array of indices in the current context

    for _ in range(max_new_tokens):
      #get the predictions
      logits, loss = self(idx) # Pass only idx to forward when generating
      #focus only on the last time step

      logits=logits[:,-1,:] # (B,C)
      #apply softmax to get the probabilities

      probs=F.softmax(logits,dim=1)

      # sample from the distribution

      idx_next=torch.multinomial(probs,num_samples=1) # (B,1)

      idx=torch.cat((idx,idx_next),dim=1) # (B,T+1)

    return idx



m=BigramLanguageModel(vocab_size)
logits,loss=m(xb,yb)
print(logits.shape,loss)

torch.Size([32, 65]) tensor(4.8786, grad_fn=<NllLossBackward0>)


In [ ]:
idx=torch.zeros((1,1),dtype=torch.long)
print(decode((m.generate(idx,max_new_tokens=10))[0].tolist()))


Sr?qP-QWkt


In [ ]:
optimizer=torch.optim.AdamW(m.parameters(),lr=1e-3)

In [ ]:
batch_size=32
for steps in range(10000):
  # sample batch of data
  xb,yb=get_batch('train')

  #evluate the loss
  logits,loss=m(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.4730143547058105


In [ ]:
print(decode((m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100))[0].tolist()))


By y a!
Caril.n y, denjxhece w illd CHAL, mer thoun s's:Conchuntilalllevise sthat dy hangilyoteng h 


## Mathematical Trick in Self-Attention

In [ ]:
torch.manual_seed(42)
a=torch.tril(torch.ones(3,3))
a/=torch.sum(a,1,keepdim=True)
b=torch.randint(0,10,(3,2)).float()
c=a@b
print("a=")
print(a)

print("---------\n")

print("b=")
print(b)

print("---------\n")

print("c=")
print(c)

print("---------\n")


a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---------

b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---------

c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
---------



In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [ ]:
#vectorized implementation of the above
# version2
wei=torch.tril(torch.ones(T,T))
wei=wei/wei.sum(1,keepdim=True)
xbow2=wei@x  # (T,T) @ (B,T,C) --->  (B,T,T) @ (B,T,C) ==> (B,T,C)

print(xbow2.shape)


torch.Size([4, 8, 2])


In [ ]:
diff = (xbow - xbow2).abs()
print(diff.max())

tensor(3.2363e-08)


In [ ]:
#version3
tril=torch.tril(torch.ones(T,T))
weights=torch.zeros((T,T))
weights=weights.masked_fill(tril==0,float('-inf'))
weights=F.softmax(weights,dim=1)
xbow3=weights@x


In [ ]:
diff = (xbow - xbow3).abs()
print(diff.max())

tensor(3.2363e-08)


In [ ]:
torch.allclose(xbow,xbow2,atol=1e-6,rtol=1e-5)

True

In [ ]:
#version 4  (self attention)

torch.manual_seed(1337)
B,T,C=4,8,32
x=torch.randn(B,T,C)

#lets see a single head perform self attention

head_size=16
key=nn.Linear(C,head_size,bias=False)
query=nn.Linear(C,head_size,bias=False)
value=nn.Linear(C,head_size,bias=False)

k=key(x)
q=query(x)

weight=q @ k.transpose(-2,-1) * (head_size**-0.5) # (B,T,16) @ (B,16,T) -----> (B,T,T)

# at initialization ,scaled attention, to maintain unit variance otherwise softmax will saturate

tril=torch.tril(torch.ones(T,T))

# weights=torch.zeros((T,T))

weights=weights.masked_fill(tril==0,float('-inf'))

weights=F.softmax(weights,dim=-1)

v=value(x)
out=weights@v
out.shape

torch.Size([4, 8, 16])

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:
k.var()

tensor(0.3164, grad_fn=<VarBackward0>)

In [ ]:
q.var()

tensor(0.3386, grad_fn=<VarBackward0>)

In [ ]:
wei.var()

tensor(0.0273)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn import Sequential
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 300
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed=384
n_head=6
n_layer=6
dropout=0.2


# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    """one head of self attention"""

    def __init__(self,head_size):
        super().__init__()
        self.key=nn.Linear(n_embed,head_size,bias=False)
        self.query=nn.Linear(n_embed,head_size,bias=False)
        self.value=nn.Linear(n_embed,head_size,bias=False)

        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))


        self.dropout=nn.Dropout(dropout)

    def forward(self,x):
        B,T,C=x.shape

        k=self.key(x) # (B,T,head_size)
        q=self.query(x) # (B,T,head_size)

        #compute the attention scores("affinity")
        wei=q@k.transpose(-2,-1) * (C**-0.5)  # (B,T,head_size) @ (B,head_size,T) ---> (B,T,T)
        wei=wei.masked_fill(self.tril[:T,:T]==0,float("-inf"))  # (B,T,T)
        wei=F.softmax(wei,dim=-1) # (B,T,T)

        wei=self.dropout(wei)
        #perform the weighted aggregation of the values
        v=self.value(x) #(B,T,head_size)

        out=wei@v # (B,T,T) @ (B,T,head_size) ---> (B,T,head_size)

        return out


class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""

    def __init__(self,num_heads,head_size):
        super().__init__()
        self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads) ])
        self.proj=nn.Linear(n_embed,n_embed)  # projection layer going back into the residual pathway
        self.dropout=nn.Dropout(dropout)
    def forward(self,x):
        out=torch.cat([h(x) for h in self.heads],dim=-1)
        out=self.proj(out)
        out=self.dropout(out)
        return out

class FeedForward(nn.Module):
    """ A simple linear layer followed by non-linearity"""

    def __init__(self,n_embed):
        super().__init__()
        self.net=Sequential(
            nn.Linear(n_embed,4*n_embed),
            nn.ReLU(),
            nn.Linear(4*n_embed,n_embed), # projection layer going back into the residual pathway
            nn.Dropout(dropout),
        )

    def forward(self,x):
        return self.net(x)



class Block(nn.Module):
    """Transformer Block : Communication followed by computation"""


    def __init__(self,n_embed,n_head):
        # n_embed : embedding dimension , n_head: number of heads we like

        super().__init__()
        head_size=n_embed//n_head
        self.sa=MultiHeadAttention(n_head,head_size)   # communication
        self.ffwd=FeedForward(n_embed) # computation

        self.ln1=nn.LayerNorm(n_embed) #Layer Normalization on the last dimension (channel dimension)
        self.ln2=nn.LayerNorm(n_embed) #Layer Normalization


    def forward(self,x):
        x=x+self.sa(self.ln1(x))   # Residual Connection
        x=x+self.ffwd(self.ln2(x)) # Residual Connection
        return x


# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table=nn.Embedding(block_size,n_embed)
        self.blocks=nn.Sequential(*[Block(n_embed,n_head) for _ in range(n_layer)])
        self.ln_f=nn.LayerNorm(n_embed)   # final layer norm
        self.lm_head=nn.Linear(n_embed,vocab_size)


    def forward(self, idx, targets=None):

        B,T=idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,n_embed)
        pos_emb=self.position_embedding_table(torch.arange(T,device=device)) # (T,C)
        x=tok_emb+pos_emb  #(B,T,C)
        x=self.blocks(x)   # (B,T,C)
        x=self.ln_f(x)     #(B,T,C)
        logits=self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):

            #crop idx to the last block size tokens
            idx_cond=idx[:,-block_size:]

            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 4.2849, val loss 4.2823
step 300: train loss 2.3183, val loss 2.3466
step 600: train loss 1.8885, val loss 2.0019
step 900: train loss 1.6504, val loss 1.8089
step 1200: train loss 1.5188, val loss 1.7069
step 1500: train loss 1.4394, val loss 1.6403
step 1800: train loss 1.3758, val loss 1.5862
step 2100: train loss 1.3274, val loss 1.5594
step 2400: train loss 1.2909, val loss 1.5387
step 2700: train loss 1.2561, val loss 1.5145
step 3000: train loss 1.2273, val loss 1.5077
step 3300: train loss 1.2016, val loss 1.4953
step 3600: train loss 1.1810, val loss 1.4953
step 3900: train loss 1.1546, val loss 1.4923
step 4200: train loss 1.1311, val loss 1.4858
step 4500: train loss 1.1064, val loss 1.4814
step 4800: train loss 1.0873, val loss 1.4943

But with prison: I with such sweet more
merely.

MENENIUS:
Sir, 'tis hi nothing. He wish'd to slep to
can cold him win his inher.

MISTRESS OVERDONE:
The last go, these weary harms all woman.
Go: I hear home to the world: w

# Trigram Language Model

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn import Sequential

from torch.optim.lr_scheduler import LambdaLR

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 200
learning_rate = 1.0
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed=384
n_head=6
n_layer=6
dropout=0.2


# ------------

torch.manual_seed(1337)

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

def custom_lr_lambda(step):
    warmup_steps=4000

    factor=(n_embed**-0.5)*min((step+1)**-0.5,step*(warmup_steps**-1.5))
    return (factor/learning_rate)

class Head(nn.Module):
    """one head of self attention"""

    def __init__(self,head_size):
        super().__init__()
        self.key=nn.Linear(n_embed,head_size,bias=False)
        self.query=nn.Linear(n_embed,head_size,bias=False)
        self.value=nn.Linear(n_embed,head_size,bias=False)

        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))


        self.dropout=nn.Dropout(dropout)

    def forward(self,x):
        B,T,C=x.shape

        k=self.key(x) # (B,T,head_size)
        q=self.query(x) # (B,T,head_size)

        #compute the attention scores("affinity")
        wei=q@k.transpose(-2,-1) * (C**-0.5)  # (B,T,head_size) @ (B,head_size,T) ---> (B,T,T)
        wei=wei.masked_fill(self.tril[:T,:T]==0,float("-inf"))  # (B,T,T)
        wei=F.softmax(wei,dim=-1) # (B,T,T)

        wei=self.dropout(wei)
        #perform the weighted aggregation of the values
        v=self.value(x) #(B,T,head_size)

        out=wei@v # (B,T,T) @ (B,T,head_size) ---> (B,T,head_size)

        return out


class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""

    def __init__(self,num_heads,head_size):
        super().__init__()
        self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads) ])
        self.proj=nn.Linear(n_embed,n_embed)  # projection layer going back into the residual pathway
        self.dropout=nn.Dropout(dropout)
    def forward(self,x):
        out=torch.cat([h(x) for h in self.heads],dim=-1)
        out=self.proj(out)
        out=self.dropout(out)
        return out

class FeedForward(nn.Module):
    """ A simple linear layer followed by non-linearity"""

    def __init__(self,n_embed):
        super().__init__()
        self.net=Sequential(
            nn.Linear(n_embed,4*n_embed),
            nn.ReLU(),
            nn.Linear(4*n_embed,n_embed), # projection layer going back into the residual pathway
            nn.Dropout(dropout),
        )

    def forward(self,x):
        return self.net(x)



class Block(nn.Module):
    """Transformer Block : Communication followed by computation"""


    def __init__(self,n_embed,n_head):
        # n_embed : embedding dimension , n_head: number of heads we like

        super().__init__()
        head_size=n_embed//n_head
        self.sa=MultiHeadAttention(n_head,head_size)   # communication
        self.ffwd=FeedForward(n_embed) # computation

        self.ln1=nn.LayerNorm(n_embed) #Layer Normalization on the last dimension (channel dimension)
        self.ln2=nn.LayerNorm(n_embed) #Layer Normalization


    def forward(self,x):
        x=x+self.sa(self.ln1(x))   # Residual Connection
        x=x+self.ffwd(self.ln2(x)) # Residual Connection
        return x


# super simple bigram model
class TrigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size*vocab_size, n_embed)
        self.position_embedding_table=nn.Embedding(block_size,n_embed)
        self.blocks=nn.Sequential(*[Block(n_embed,n_head) for _ in range(n_layer)])
        self.ln_f=nn.LayerNorm(n_embed)   # final layer norm
        self.lm_head=nn.Linear(n_embed,vocab_size)


    def forward(self, idx, targets=None):

        B,T=idx.shape

        if T<2:
            logits=torch.zeros(B,T,vocab_size,device=idx.device)
        else :
            prev1=idx[:,:-2]   # (B,T-2)
            prev2=idx[:,1:-1]  # (B,T-2)
            pair_idx=prev1*vocab_size+prev2
            tok_emb = self.token_embedding_table(pair_idx) # (B,T-2,n_embed)
            pad=torch.zeros(B,2,n_embed,device=idx.device)
            tok_emb=torch.cat([pad,tok_emb],dim=1)
            pos_emb=self.position_embedding_table(torch.arange(T,device=device)) # (T,C = n_embed)
            x=tok_emb+pos_emb  #(B,T,C)
            x=self.blocks(x)   # (B,T,C)
            x=self.ln_f(x)     #(B,T,C)
            logits=self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):

            #crop idx to the last block size tokens
            idx_cond=idx[:,-block_size:]

            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = TrigramLanguageModel()
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler=LambdaLR(optimizer,lr_lambda=custom_lr_lambda)
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    scheduler.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 4.3067, val loss 4.3075
step 200: train loss 3.1677, val loss 3.2125
step 400: train loss 2.7698, val loss 2.8392
step 600: train loss 2.6064, val loss 2.7259
step 800: train loss 2.5395, val loss 2.6805
step 1000: train loss 2.4855, val loss 2.6362
step 1200: train loss 2.4024, val loss 2.5804
step 1400: train loss 2.3106, val loss 2.5233
step 1600: train loss 2.2311, val loss 2.4545
step 1800: train loss 2.1579, val loss 2.3943
step 2000: train loss 2.0951, val loss 2.3588
step 2200: train loss 2.0420, val loss 2.3170
step 2400: train loss 1.9922, val loss 2.2948
step 2600: train loss 1.9453, val loss 2.2695
step 2800: train loss 1.9054, val loss 2.2702
step 3000: train loss 1.8571, val loss 2.2403
step 3200: train loss 1.8221, val loss 2.2492
step 3400: train loss 1.7900, val loss 2.2356
step 3600: train loss 1.7534, val loss 2.2462
step 3800: train loss 1.7177, val loss 2.2413
step 4000: train loss 1.6832, val loss 2.2492
step 4200: train loss 1.6406, val loss 2.